In [2]:
%matplotlib qt

from numpy import *
import scipy.linalg as linalg
from matplotlib.pyplot import *
import pickle
from quadcopter.util import *

rcParams['figure.figsize'] = (10, 6)
rcParams['figure.dpi'] = 160
rcParams['pgf.rcfonts'] = False
rcParams['lines.linewidth'] = 1.2
rcParams['figure.autolayout'] = True

# Replace these paths with the appropriate directories
plot_dir = r'C:\example\plots'
result_dir = r'C:\example\simulation_results'


colors = {
    'reference': 'black',
    'ekf': 'red',
    'mhe1': ['green', 'olive', 'jungle green'],
    'mhe2': ['blue', 'amethyst', 'water blue']
}


def save_plot(plot_name):
    savefig(f"{plot_dir}\\{plot_name}.pgf")


def to_deg(rad):
    return rad / np.pi * 180.0


def load_results(file_name):
    complete_path = f"{result_dir}\\{file_name}.pmr"
    with open(complete_path, "rb") as f:
        return pickle.load(f)['results']
    

class ExperimentData:
    def __init__(self, ex_id, raw):
        self.ex_id = ex_id
        self.raw = raw
    
    @property
    def time(self):
        return self.raw['time']
   
    @property 
    def solver(self):
        return self.raw['Solver'].T
    
    @property
    def observer(self):
        return self.raw['Observer'][:, :4].T
    
    @property
    def covariance(self):
        return self.raw['Observer'][:, 4:].T
    
    @property
    def measurement(self):
        return self.raw['Model'].T
    
    
def load_experiment(ex_id, resample_steps=None, start_time=None, end_time=None) -> ExperimentData:
    raw = load_results(ex_id)
    
    start_index = None
    end_index = None
    
    if start_time is not None:
        start_index = time_to_index(start_time, raw['time'])
        
    if end_time is not None:
        end_index = time_to_index(end_time, raw['time'])
    
    raw['time'] = raw['time'][start_index:end_index:resample_steps]
    raw['Solver'] = raw['Solver'][start_index:end_index:resample_steps]
    raw['Observer'] = raw['Observer'][start_index:end_index:resample_steps]
    raw['Model'] = raw['Model'][start_index:end_index:resample_steps]
    return ExperimentData(ex_id, raw)

In [2]:
ex1 = load_experiment('511')
ex2 = load_experiment('512')
combined_plot = True

figure()
plot(ex1.time, ex1.measurement[0])

if combined_plot:
    figure("Combined plot", figsize=(18, 10))

for i in range(4):
    if combined_plot:
        subplot(421 + 2*i)
    else:
        figure(f"State {i+1} observers")
        
    plot(ex1.time, ex1.solver[i], '--', c="blue")
    plot(ex1.time, ex1.observer[i], c="red")
    plot(ex1.time, ex2.observer[i], c="green")
    legend(["reference", ex1.ex_id, ex2.ex_id])
    title(f"State {i+1} observers")
    grid()
    
    if combined_plot:
        subplot(421 + 2*i + 1)
    else:
        figure(f"State {i+1} errors")
        
    plot(ex1.time, ex1.observer[i] - ex1.solver[i], c="red")
    plot(ex1.time, ex2.observer[i] - ex2.solver[i], c="green")
    mse1 = sum([e**2 for e in ex1.observer[i] - ex1.solver[i]]) / ex1.time.size
    mse2 = sum([e**2 for e in ex2.observer[i] - ex2.solver[i]]) / ex2.time.size
    legend([f"{ex1.ex_id} - mse={mse1}", f"{ex2.ex_id} - mse={mse2}"])
    title(f"State {i+1} errors")
    grid()

show()

In [9]:
ekf_exs = [load_experiment('511'), load_experiment('531'), load_experiment('541')]
mhe_exs = [load_experiment('512'), load_experiment('532'), load_experiment('542')]
ref = ekf_exs[0]
rows = len(ekf_exs)

figure(figsize=(6.5, 6))
for i in range(rows):
    subplot(rows, 2, 2*i + 1)
    ref_line, = plot(ref.time, to_deg(ref.solver[2]), color=colors['reference'])
    ekf_line, = plot(ref.time, to_deg(ekf_exs[i].observer[2]), color=colors['ekf'], linestyle='--')
    mhe_line, = plot(ref.time, to_deg(mhe_exs[i].observer[2]), color=colors['mhe1'][0], linestyle='--')
    xlabel('$t$ (s)')
    ylabel('$\\theta$ (deg)')
    grid()
    
    subplot(rows, 2, 2*i + 2)
    plot(ref.time, to_deg(ekf_exs[i].observer[2] - ref.solver[2]), color=colors['ekf'], linestyle='--')
    plot(ref.time, to_deg(mhe_exs[i].observer[2] - ref.solver[2]), color=colors['mhe1'][0], linestyle='--')
    xlabel('$t$ (s)')
    ylabel('$\\mathit{\\Delta\\theta}$ (deg)')
    grid()

figlegend((ref_line, ekf_line, mhe_line), ('Referenz', 'EKF', 'MHE N=8'))
tight_layout()
save_plot('step_width')
#tikz_save('plots/step_width.tex', figureheight='\\figureheight', figurewidth='\\figurewidth')
show()

C:\dev\Python\studienarbeit\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['lmodern'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [3]:
figure(figsize=(4, 2.5))
mhe_scipy_line, = plot([1, 4, 8], [7.73, 52.48, 145.78], color='b', marker='o')
mhe_rti_line, = plot([1, 4, 8, 40], [2.37, 3.16, 3.56, 12.42], color='g', marker='o')
abtastzeit_line = axhline(5, color=colors['reference'], linestyle='--')
ekf_line = axhline(0.2, color=colors['ekf'], linestyle='--')
yscale('log')
xlabel('Horizontweite')
ylabel('Rechenzeit Beobachter (ms)')
figlegend((abtastzeit_line, ekf_line, mhe_scipy_line, mhe_rti_line),
          ('Abtastzeit', 'EKF', 'MHE-SciPy', 'MHE-RTI'))

save_plot('performance')
#tikz_save('plots/performance.tex', figureheight='\\figureheight', figurewidth='\\figurewidth')
show()

C:\dev\Python\studienarbeit\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['lmodern'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [3]:
ekf = load_experiment('611', end_time=8.0)
mhe1 = load_experiment('612', end_time=8.0)
mhe16 = load_experiment('613', end_time=8.0)
ref = ekf

figure(figsize=(6.5, 8))

subplot(4, 2, 1)
ref_line, = plot(ref.time, ref.solver[0], color=colors['reference'])
ekf_line, = plot(ref.time, ekf.observer[0], color=colors['ekf'], linestyle='--')
mhe1_line, = plot(ref.time, mhe1.observer[0], color=colors['mhe1'][0], linestyle='--')
mhe16_line, = plot(ref.time, mhe16.observer[0], color=colors['mhe1'][1], linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$r$ (m)')

subplot(4, 2, 2)
plot(ref.time, ekf.observer[0] - ref.solver[0], color=colors['ekf'], linestyle='--')
plot(ref.time, mhe1.observer[0] - ref.solver[0], color=colors['mhe1'][0], linestyle='--')
plot(ref.time, mhe16.observer[0] - ref.solver[0], color=colors['mhe1'][1], linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\mathit{\\Delta r}$ (m)')

subplot(4, 2, 3)
plot(ref.time, ref.solver[1], color=colors['reference'])
plot(ref.time, ekf.observer[1], color=colors['ekf'], linestyle='--')
plot(ref.time, mhe1.observer[1], color=colors['mhe1'][0], linestyle='--')
plot(ref.time, mhe16.observer[1], color=colors['mhe1'][1], linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$v$ (m/s)')

subplot(4, 2, 4)
plot(ref.time, ekf.observer[1] - ref.solver[1], color=colors['ekf'], linestyle='--')
plot(ref.time, mhe1.observer[1] - ref.solver[1], color=colors['mhe1'][0], linestyle='--')
plot(ref.time, mhe16.observer[1] - ref.solver[1], color=colors['mhe1'][1], linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\mathit{\\Delta v}$ (m/s)')

subplot(4, 2, 5)
plot(ref.time, to_deg(ref.solver[2]), color=colors['reference'])
plot(ref.time, to_deg(ekf.observer[2]), color=colors['ekf'], linestyle='--')
plot(ref.time, to_deg(mhe1.observer[2]), color=colors['mhe1'][0], linestyle='--')
plot(ref.time, to_deg(mhe16.observer[2]), color=colors['mhe1'][1], linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\theta$ (deg)')

subplot(4, 2, 6)
plot(ref.time, to_deg(ekf.observer[2] - ref.solver[2]), color=colors['ekf'], linestyle='--')
plot(ref.time, to_deg(mhe1.observer[2] - ref.solver[2]), color=colors['mhe1'][0], linestyle='--')
plot(ref.time, to_deg(mhe16.observer[2] - ref.solver[2]), color=colors['mhe1'][1], linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\mathit{\\Delta\\theta}$ (deg)')

subplot(4, 2, 7)
plot(ref.time, to_deg(ref.solver[3]), color=colors['reference'])
plot(ref.time, to_deg(ekf.observer[3]), color=colors['ekf'], linestyle='--')
plot(ref.time, to_deg(mhe1.observer[3]), color=colors['mhe1'][0], linestyle='--')
plot(ref.time, to_deg(mhe16.observer[3]), color=colors['mhe1'][1], linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\omega$ (deg/s)')

subplot(4, 2, 8)
plot(ref.time, to_deg(ekf.observer[3] - ref.solver[3]), color=colors['ekf'], linestyle='--')
plot(ref.time, to_deg(mhe1.observer[3] - ref.solver[3]), color=colors['mhe1'][0], linestyle='--')
plot(ref.time, to_deg(mhe16.observer[3] - ref.solver[3]), color=colors['mhe1'][1], linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\mathit{\\Delta\\omega}$ (deg/s)')

figlegend((ref_line, ekf_line, mhe1_line, mhe16_line),
          ('Referenz', 'EKF', 'MHE N=1', 'MHE N=16'))

tight_layout()

save_plot('medium_initial_error')
#tikz_save('plots/medium_initial_error.tex', figureheight='\\figureheight', figurewidth='\\figurewidth')
show()

C:\dev\Python\studienarbeit\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['lmodern'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [7]:
ekf = load_experiment('611', end_time=8)
mhe1 = load_experiment('612', end_time=8)
mhe16 = load_experiment('613', end_time=8)
ref = ekf

figure(figsize=(4, 2.5))

threshold = 0.5

ekf_error_norm = linalg.norm(ekf.observer - ref.solver, axis=0)
mhe1_error_norm = linalg.norm(mhe1.observer - ref.solver, axis=0)
mhe8_error_norm = linalg.norm(mhe16.observer - ref.solver, axis=0)
ekf_line, = plot(ref.time, ekf_error_norm, color=colors['ekf'])
mhe1_line, = plot(ref.time, mhe1_error_norm, color=colors['mhe1'][0])
mhe8_line, = plot(ref.time, mhe8_error_norm, color=colors['mhe1'][1])

ekf_convergence_index = argmax([all(ekf_error_norm[i:] < threshold) for i in range(ekf_error_norm.size)])
mhe1_convergence_index = argmax([all(mhe1_error_norm[i:] < threshold) for i in range(mhe1_error_norm.size)])
mhe8_convergence_index = argmax([all(mhe8_error_norm[i:] < threshold) for i in range(mhe8_error_norm.size)])

ekf_conv_time = ref.time[ekf_convergence_index]
mhe1_conv_time = ref.time[mhe1_convergence_index]
mhe8_conv_time = ref.time[mhe8_convergence_index]

print(f"EKF: {ekf_conv_time}, MHE1: {mhe1_conv_time}, MHE8: {mhe8_conv_time}")

axvline(ekf_conv_time, linestyle='--', color=colors['ekf'])
axvline(mhe1_conv_time, linestyle='--', color=colors['mhe1'][0])
axvline(mhe8_conv_time, linestyle='--', color=colors['mhe1'][1])

axhline(threshold, color='black', linestyle='--')
axhline(0, color='black')

xlabel('$t$ (s)')
ylabel('$\\Vert \mathit{\Delta x} \\Vert$')

figlegend((ekf_line, mhe1_line, mhe8_line),
          ('EKF', 'MHE N=1', 'MHE N=16'))

save_plot('medium_convergence')
show()

EKF: 1.405999999999956, MHE1: 1.3999999999999566, MHE8: 1.159999999999983


C:\dev\Python\studienarbeit\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['lmodern'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [8]:
ekf = load_experiment('621', end_time=8)
mhe1 = load_experiment('622', end_time=8)
mhe16 = load_experiment('623', end_time=8)
ref = ekf

figure(figsize=(4, 2.5))

threshold = 0.5

ekf_error_norm = linalg.norm(ekf.observer - ref.solver, axis=0)
mhe1_error_norm = linalg.norm(mhe1.observer - ref.solver, axis=0)
mhe8_error_norm = linalg.norm(mhe16.observer - ref.solver, axis=0)
ekf_line, = plot(ref.time, ekf_error_norm, color=colors['ekf'])
mhe1_line, = plot(ref.time, mhe1_error_norm, color=colors['mhe1'][0])
mhe8_line, = plot(ref.time, mhe8_error_norm, color=colors['mhe1'][1])

ekf_convergence_index = argmax([all(ekf_error_norm[i:] < threshold) for i in range(ekf_error_norm.size)])
mhe1_convergence_index = argmax([all(mhe1_error_norm[i:] < threshold) for i in range(mhe1_error_norm.size)])
mhe8_convergence_index = argmax([all(mhe8_error_norm[i:] < threshold) for i in range(mhe8_error_norm.size)])

ekf_conv_time = ref.time[ekf_convergence_index]
mhe1_conv_time = ref.time[mhe1_convergence_index]
mhe8_conv_time = ref.time[mhe8_convergence_index]

print(f"EKF: {ekf_conv_time}, MHE1: {mhe1_conv_time}, MHE8: {mhe8_conv_time}")

#axvline(ekf_conv_time, linestyle='--', lw=1.2, color=colors['ekf'])
#axvline(mhe1_conv_time, linestyle='--', lw=1.2, color=colors['mhe1'][0])
axvline(mhe8_conv_time, linestyle='--', color=colors['mhe1'][1])

axhline(threshold, color='black', linestyle='--')
axhline(0, color='black')

xlabel('$t$ (s)')
ylabel('$\\Vert \mathit{\Delta x} \\Vert$')

figlegend((ekf_line, mhe1_line, mhe8_line),
          ('EKF', 'MHE N=1', 'MHE N=16'))

save_plot('large_convergence')
show()

EKF: 0.0, MHE1: 0.0, MHE8: 1.1299999999999863


C:\dev\Python\studienarbeit\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['lmodern'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [6]:
ekf = load_experiment('621', end_time=8)
mhe1 = load_experiment('622', end_time=8)
mhe16 = load_experiment('623', end_time=8)
ref = ekf

figure(figsize=(6.5, 8))

subplot(4, 2, 1)
ref_line, = plot(ref.time, ref.solver[0], color=colors['reference'])
ekf_line, = plot(ref.time, ekf.observer[0], color=colors['ekf'], linestyle='--')
mhe1_line, = plot(ref.time, mhe1.observer[0], color=colors['mhe1'][0], linestyle='--')
mhe16_line, = plot(ref.time, mhe16.observer[0], color=colors['mhe1'][1], linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$r$ (m)')

subplot(4, 2, 2)
plot(ref.time, ekf.observer[0] - ref.solver[0], color=colors['ekf'], linestyle='--')
plot(ref.time, mhe1.observer[0] - ref.solver[0], color=colors['mhe1'][0], linestyle='--')
plot(ref.time, mhe16.observer[0] - ref.solver[0], color=colors['mhe1'][1], linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\mathit{\\Delta r}$ (m)')

subplot(4, 2, 3)
plot(ref.time, ref.solver[1], color=colors['reference'])
plot(ref.time, ekf.observer[1], color=colors['ekf'], linestyle='--')
plot(ref.time, mhe1.observer[1], color=colors['mhe1'][0], linestyle='--')
plot(ref.time, mhe16.observer[1], color=colors['mhe1'][1], linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$v$ (m/s)')

subplot(4, 2, 4)
plot(ref.time, ekf.observer[1] - ref.solver[1], color=colors['ekf'], linestyle='--')
plot(ref.time, mhe1.observer[1] - ref.solver[1], color=colors['mhe1'][0], linestyle='--')
plot(ref.time, mhe16.observer[1] - ref.solver[1], color=colors['mhe1'][1], linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\mathit{\\Delta v}$ (m/s)')

subplot(4, 2, 5)
plot(ref.time, to_deg(ref.solver[2]), color=colors['reference'])
plot(ref.time, to_deg(ekf.observer[2]), color=colors['ekf'], linestyle='--')
plot(ref.time, to_deg(mhe1.observer[2]), color=colors['mhe1'][0], linestyle='--')
plot(ref.time, to_deg(mhe16.observer[2]), color=colors['mhe1'][1], linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\theta$ (deg)')

subplot(4, 2, 6)
plot(ref.time, to_deg(ekf.observer[2] - ref.solver[2]), color=colors['ekf'], linestyle='--')
plot(ref.time, to_deg(mhe1.observer[2] - ref.solver[2]), color=colors['mhe1'][0], linestyle='--')
plot(ref.time, to_deg(mhe16.observer[2] - ref.solver[2]), color=colors['mhe1'][1], linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\mathit{\\Delta\\theta}$ (deg)')

subplot(4, 2, 7)
plot(ref.time, to_deg(ref.solver[3]), color=colors['reference'])
plot(ref.time, to_deg(ekf.observer[3]), color=colors['ekf'], linestyle='--')
plot(ref.time, to_deg(mhe1.observer[3]), color=colors['mhe1'][0], linestyle='--')
plot(ref.time, to_deg(mhe16.observer[3]), color=colors['mhe1'][1], linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\omega$ (deg/s)')

subplot(4, 2, 8)
plot(ref.time, to_deg(ekf.observer[3] - ref.solver[3]), color=colors['ekf'], linestyle='--')
plot(ref.time, to_deg(mhe1.observer[3] - ref.solver[3]), color=colors['mhe1'][0], linestyle='--')
plot(ref.time, to_deg(mhe16.observer[3] - ref.solver[3]), color=colors['mhe1'][1], linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\mathit{\\Delta\\omega}$ (deg/s)')

figlegend((ref_line, ekf_line, mhe1_line, mhe16_line),
          ('Referenz', 'EKF', 'MHE N=1', 'MHE N=16'))

tight_layout()

save_plot('large_initial_error')
#tikz_save('plots/large_initial_error.tex', figureheight='\\figureheight', figurewidth='\\figurewidth')
show()

C:\dev\Python\studienarbeit\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['lmodern'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))
